In [1]:
from numpy import char
from datetime import date
import pandas as pd
import numpy as np
import time
pd.set_option('display.float_format', str)

In [2]:
# Lineitem

l_columnnames = ["L_ORDERKEY", "L_PARTKEY", "L_SUPPKEY", "L_LINENUMBER", "L_QUANTITY", "L_EXTENDEDPRICE", "L_DISCOUNT", "L_TAX",
                "L_RETURNFLAG", "L_LINESTATUS", "L_SHIPDATE", "L_COMMITDATE", "L_RECEIPTDATE", "L_SHIPINSTRUCT", "L_SHIPMODE", "L_COMMENT"]

for i in range(len(l_columnnames)):
    l_columnnames[i] = l_columnnames[i].lower()

l_data_types = {
    'l_orderkey': int,
    'l_partkey': int,
    'l_suppkey': int,
    'l_linenumber': int,
    'l_quantity': float,
    'l_extendedprice': float,
    'l_discount': float,
    'l_tax': float,
    'l_returnflag': str,
    'l_linestatus': str,
    'l_shipinstruct': str,
    'l_shipmode': str,
    'l_comment': str
}

l_parse_dates = ['l_shipdate', 'l_commitdate', 'l_receiptdate']

# Don't set indexes, as we can't access them with Pandas selection!
lineitem = pd.read_table("../data_storage/lineitem.tbl.csv", sep="|", names=l_columnnames, dtype=l_data_types, parse_dates=l_parse_dates)

# Order

o_columnnames = ["O_ORDERKEY", "O_CUSTKEY", "O_ORDERSTATUS", "O_TOTALPRICE", "O_ORDERDATE", "O_ORDERPRIORITY", "O_CLERK", "O_SHIPPRIORITY", "O_COMMENT"]

for i in range(len(o_columnnames)):
    o_columnnames[i] = o_columnnames[i].lower()
    
o_data_types = {
    'o_orderkey': int,
    'o_custkey': int,
    'o_orderstatus': str,
    'o_totalprice': float,
    'o_orderpriority': str,
    'o_clerk': str,
    'o_shippriority': int,
    'o_comment': str
}

o_parse_dates = ['o_orderdate']

# Don't set indexes, as we can't access them with Pandas selection!
orders = pd.read_table("../data_storage/orders.tbl.csv", sep="|", names=o_columnnames, dtype=o_data_types, parse_dates=o_parse_dates)

# Customer

c_columnnames = ["C_CUSTKEY", "C_NAME", "C_ADDRESS", "C_NATIONKEY", "C_PHONE", "C_ACCTBAL", "C_MKTSEGMENT", "C_COMMENT"]

for i in range(len(c_columnnames)):
    c_columnnames[i] = c_columnnames[i].lower()
    
c_data_types = {
    'c_custkey': int,
    'c_name': str,
    'c_address': str,
    'c_nationkey': int,
    'c_phone': str,
    'c_acctbal': float,
    'c_mktsegment': str,
    'c_comment': str
}

c_parse_dates = []

# Don't set indexes, as we can't access them with Pandas selection!
customer = pd.read_table("../data_storage/customer.tbl.csv", sep="|", names=c_columnnames, dtype=c_data_types, parse_dates=c_parse_dates)

# Part

p_columnnames = ["P_PARTKEY", "P_NAME", "P_MFGR", "P_BRAND", "P_TYPE", "P_SIZE", "P_CONTAINER", "P_RETAILPRICE", "P_COMMENT"]

for i in range(len(p_columnnames)):
    p_columnnames[i] = p_columnnames[i].lower()
    
p_data_types = {
    'p_partkey': int, 
    'p_name': str,
    'p_mfgr': str,
    'p_brand': str,
    'p_type': str,
    'p_size': int,
    'p_container': str,
    'p_retailprice': float,
    'p_comment': str
}

p_parse_dates = []

# Don't set indexes, as we can't access them with Pandas selection!
part = pd.read_table("../data_storage/part.tbl.csv", sep="|", names=p_columnnames, dtype=p_data_types, parse_dates=p_parse_dates)

# Nation

n_columnnames = ["N_NATIONKEY", "N_NAME", "N_REGIONKEY", "N_COMMENT"]

for i in range(len(n_columnnames)):
    n_columnnames[i] = n_columnnames[i].lower()
    
n_data_types = {
    'n_nationkey': int,
    'n_name': str,
    'n_regionkey': int,
    'n_comment': str,
}

n_parse_dates = []

# Don't set indexes, as we can't access them with Pandas selection!
nation = pd.read_table("../data_storage/nation.tbl.csv", sep="|", names=n_columnnames, dtype=n_data_types, parse_dates=n_parse_dates)

# Supplier

s_columnnames = ["S_SUPPKEY", "S_NAME", "S_ADDRESS", "S_NATIONKEY", "S_PHONE", "S_ACCTBAL", "S_COMMENT"]

for i in range(len(s_columnnames)):
    s_columnnames[i] = s_columnnames[i].lower()

s_data_types = {
    's_suppkey': int,
    's_name': str,
    's_address': str,
    's_nationkey': int,
    's_phone': str,
    's_acctbal': float,
    's_comment': str
}

s_parse_dates = []

# Don't set indexes, as we can't access them with Pandas selection!
# Don't set indexes, as we can't access them with Pandas selection!
supplier = pd.read_table("../data_storage/supplier.tbl.csv", sep="|", names=s_columnnames, dtype=s_data_types, parse_dates=s_parse_dates)

In [3]:
df_filter_1 = nation[nation.n_name == "SAUDI ARABIA"]
df_filter_2 = supplier
df_merge_1 = df_filter_2.merge(df_filter_1, left_on='s_nationkey', right_on='n_nationkey')
# Correct

In [4]:
df_filter_3 = lineitem[lineitem.l_receiptdate > lineitem.l_commitdate]
df_merge_2 = df_filter_3.merge(df_merge_1, left_on='l_suppkey', right_on='s_suppkey')
# Correct

In [23]:
df_filter_4 = lineitem[lineitem.l_receiptdate > lineitem.l_commitdate]


# First we inner-join to handle the condition, and we create a new dataframe of all l_orderkeys that match

inner_cond = df_merge_2.merge(df_filter_4, left_on='l_orderkey', right_on='l_orderkey', how='inner')
inner_cond = inner_cond[inner_cond.l_suppkey_x != inner_cond.l_suppkey_y]['l_orderkey']

# Then we anti-join this with the original dataframe

df_merge_3 = df_merge_2.merge(inner_cond, left_on='l_orderkey', right_on='l_orderkey', how='outer', indicator=True)
df_merge_3 = df_merge_3[df_merge_3._merge == "left_only"]

0        1
1        1
2        1
4    28132
6    40420
Name: l_orderkey, dtype: int64


In [6]:
# df_filter_5 is orders
df_filter_5 = orders[orders.o_orderstatus == 'F']
df_filter_5 = df_filter_5[['o_orderkey', 'o_custkey', 'o_orderstatus', 'o_totalprice', 'o_orderdate', 'o_orderpriority', 'o_clerk', 'o_shippriority', 'o_comment']]

In [24]:
df_sort_1 = df_merge_3.sort_values(by=['l_orderkey'], ascending=[True])
df_sort_1 = df_sort_1[['s_name', 'l_suppkey', 'l_orderkey']]
df_merge_4 = df_filter_5.merge(df_sort_1, left_on=['o_orderkey'], right_on=['l_orderkey'])
df_merge_4 = df_merge_4[['s_name', 'l_suppkey', 'l_orderkey', 'o_orderkey']]
df_filter_6 = lineitem[['l_orderkey', 'l_partkey', 'l_suppkey', 'l_linenumber', 'l_quantity', 'l_extendedprice', 'l_discount', 'l_tax', 'l_returnflag', 'l_linestatus', 'l_shipdate', 'l_commitdate', 'l_receiptdate', 'l_shipinstruct', 'l_shipmode', 'l_comment']]

# Swap from semi-join to using an inner join
# Use inner-join to perform the condition
inner_cond = df_merge_4.merge(df_filter_6, left_on=["o_orderkey"], right_on=["l_orderkey"], how="inner")
inner_cond = inner_cond[inner_cond.l_suppkey_y != inner_cond.l_suppkey_x]['o_orderkey']

# Then semi-join on the data to complete the process
# We have already limited the columns, so just do ".isin" for the series
df_merge_5 = df_merge_4[df_merge_4.o_orderkey.isin(inner_cond)]

df_merge_5 = df_merge_5[['s_name']]
df_sort_2 = df_merge_5.sort_values(by=['s_name'], ascending=[True])
df_sort_2 = df_sort_2[['s_name']]
df_group_1 = df_sort_2 \
    .groupby(['s_name'], sort=False) \
    .agg(
        numwait=("s_name", "count"),
    )
df_group_1 = df_group_1[['numwait']]
df_sort_3 = df_group_1.sort_values(by=['numwait', 's_name'], ascending=[False, True])
df_sort_3 = df_sort_3[['numwait']]
df_limit_1 = df_sort_3[['numwait']]
df_limit_1 = df_limit_1.head(100)
# return df_limit_1

3     1344
6     1699
7     2147
8     2147
10    2147
Name: o_orderkey, dtype: int64
Index(['s_name', 'l_suppkey', 'l_orderkey', 'o_orderkey'], dtype='object')


In [25]:
df_limit_1

,numwait
s_name,
Supplier#000002829,20
Supplier#000005808,18
Supplier#000000262,17
Supplier#000000496,17
Supplier#000002160,17
...,...
Supplier#000001916,12
Supplier#000001925,12
Supplier#000002039,12
